In [1]:
def set_proxy():
    import os
    cache_dir = "/home/tx/workspace/cache"  # 替换为你想要的缓存目录的路径
    os.environ['TOKENIZERS_PARALLELISM'] = 'true'
    # 代理
    os.environ['http_proxy'] = 'http://127.0.0.1:7890'
    os.environ['https_proxy'] = 'http://127.0.0.1:7890'
    os.environ['no_proxy'] = '127.0.0.1,localhost'
    os.environ['HTTP_PROXY'] = 'http://127.0.0.1:7890'
    os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:7890'
    os.environ['NO_PROXY'] = '127.0.0.1,localhost'
    return
set_proxy()

In [2]:
import torch

In [3]:
#cache_dir = "/home/tx/workspace/cache"  # 替换为你想要的缓存目录的路径
from transformers import AutoModelForCausalLM,AutoTokenizer
from transformers import AutoConfig, LlamaConfig 
cache_dir = "/home/tx/workspace/cache" 
model_name = "bigscience/bloom-3b" # bigscience/bloom-3b
#model_name = "ChocoWu/nextgpt_7b_tiva_v0"#"liuhaotian/llava-v1.5-7b"
original_model = AutoModelForCausalLM.from_pretrained(model_name,trust_remote_code=True, cache_dir=cache_dir)
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=cache_dir)

peft_parameters_path = "../../saved_model/ai_detector_peft_final/"
#original_model.

In [4]:
from peft import PeftModel
model = PeftModel.from_pretrained(original_model,model_id="./ai_detector_peft/")


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
For effortless bug reporting copy-paste your error into this form: https://docs.google.com/forms/d/e/1FAIpQLScPB8emS3Thkp66nvqwmjTEgxp8Y9ufuWTzFyr9kJ5AoI47dQ/viewform?usp=sf_link
CUDA SETUP: CUDA runtime path found: /usr/local/cuda-11.7/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 5.2
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /home/tx/.conda/envs/llava/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cuda117_nocublaslt.so...


/home/tx/.conda/envs/llava/lib/python3.11/site-packages/bitsandbytes/cuda_setup/paths.py:93: UserWarning: /home/tx/.conda/envs/llava did not contain libcudart.so as expected! Searching further paths...
  warn(


In [5]:
def generate_prompt_ids(feature_text,tokenizer,max_length):
    prompt1=  f"""
        read the following text, understand its style and words choices:
    """
    front_ids = tokenizer.encode(prompt1)
    feature_text_ds = tokenizer.encode(feature_text)
    prompt2 = """
        is this text generated by AI?
    """
    after_ids = tokenizer.encode(prompt2)

     
    if(len(front_ids) + len(feature_text_ds) + len(after_ids) > max_length):
        
        ellipsis = tokenizer.encode('...')
        feature_len = max_length - len(front_ids) - len(after_ids) - len(ellipsis)
        head_part_feature = feature_text_ds[ : int(feature_len / 2)]
        tail_part_feature = feature_text_ds[ (0 - int(feature_len / 2)): ]

        input_ids = front_ids + head_part_feature + ellipsis + tail_part_feature + after_ids
    else:
        input_ids = front_ids + feature_text_ds + after_ids
    final_text = tokenizer.decode(input_ids)
    return input_ids,final_text            

In [6]:
max_length = 256
yes_label = "yes, the text is generated"
no_label = "no, the text is written by students"
len_yes_label = len(tokenizer.encode(yes_label))
len_no_label = len(tokenizer.encode(no_label))
            
def generate(feature_text,model_,tokenizer_):
    input_ids,_ = generate_prompt_ids(feature_text,tokenizer,max_length - max(len_yes_label,len_no_label))
    input_ids = torch.LongTensor([input_ids]).to(model_.device)
    #model_.disable_adapter()
    # 生成文本
    inputs = {"input_ids":input_ids, "max_length":max_length, 
              "num_beams":5, "no_repeat_ngram_size":2,
             "top_k":50, "top_p":0.95}
    with torch.no_grad():
        output = model_.generate(**inputs)
    # 将生成的token解码成文本
    generated_text = tokenizer_.decode(output[0], skip_special_tokens=True)
    return generated_text
def pred(r):
    id  = r['id']
    
    text = r['text']
    final_pred =  generate(text,model_=model,tokenizer_=tokenizer)
    #print(final_pred)
    final_preds = final_pred.strip().split('\n')
    #print(final_preds)
    label_pred = final_preds[-1]
    label_pred = label_pred.strip()
    return label_pred

In [7]:
model = model.cuda()

In [8]:
import pandas as pd
folder = "./data" # /kaggle/input/llm-detect-ai-generated-text
train_data = pd.read_csv(folder+"/train_essays.csv")

In [11]:
final_result = {
    "id":[],
    "generated":[],
    "final_pred":[]
}

In [13]:
from tqdm.auto import tqdm
for r in tqdm(train_data.iterrows()):
    label_pred = pred(r[1])
    # [id,label,final_pred,text,label_pred]
    #   0.  1.    2         3.    4
    final_result['id'].append(r[1]['id'])
    final_result['generated'].append(r[1]['generated'])
    final_result['final_pred'].append(label_pred)
    final_result['final_pred'].append(r[1]['text'])


0it [00:00, ?it/s]